In [157]:
from Bio.PDB import PDBParser
from Bio import pairwise2
import pandas as pd

In [158]:
from prism_parser_helper import read_from_prism
from pdb_to_prism import pdb_renumb
from prism2mutfile import residue_files_func, combined_mut_func, pipeline_combined_mut_func

In [159]:
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M', 'BNG': 'X'}

AA=['A','C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
       'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [160]:
pdb_p = PDBParser()
structure = pdb_p.get_structure('test', '/sbinlab/panf/Cartesian_pipeline_runs/Nicolas/1k62/1k62.pdb')
model = structure[0]
all_data = {}
nchains = []
for chain in model:
    nchain = str(chain).split('=')[1][0] # Gets chain ID from the string "<Chain id=A>"
    nchains.append(nchain)
    sequence = []
    residues = []
    i = 0
    for res in chain:
        residue = str(res).split() 
        sequence.append(d3to1[residue[1]]) # Gets residue 1-letter code from [Residue ILE het=  resseq=20 icode= >]
        residues.append(int(residue[3].split('=')[1])) # Gets numeration
        i += 1
    all_data[nchain] = [sequence, residues]
seq_A = "".join(all_data[nchains[0]][0])
seq_B = "".join(all_data[nchains[1]][0])

In [161]:
seq_A

'GAVDPIMEKFNASIAYDRHLWEVDVQGSKAYSRGLEKAGLLTKAEMDQILHGLDKVAEEWAQGTFKLNSNDEDIHTANERRLKELIGATAGKLHTGRSRNDQVVTDLRLWMRQTCSTLSGLLWELIRTMVDRAEAERDVLFPGYTHLQRAQPIRWSHWILSHAVALTRDSERLLEVRKRINVLPLGSGAIAGNPLGVDRELLRAELNFGAITLNSMDATSERDFVAEFLFWRSLCMTHLSRMAEDLILYCTKEFSFVQLSDAYSTGSSLMPRKKNPDSLELIRSKAGRVFGRCAGLLMTLKGLPSTYNKDLQEDKEAVFEVSDTMSAVLQVATGVISTLQIHQENMGQALSPDMLATDLAYYLVRKGMPFRQAHEASGKAVFMAETKGVALNQLSLQELQTISPLFSGDVICVWDYRHSVEQYGALGGTARSSVDWQIRQVRALLQAQQA'

In [162]:
seq_B

'GKLWGGRFVGAVDPIMEKFNASIAYDRHLWEVDVQGSKAYSRGLEKAGLLTKAEMDQILHGLDKVAEEWAQGTFKLNSNDEDIHTANERRLKELIGATAGKLHTGRSRNDQVVTDLRLWMRQTCSTLSGLLWELIRTMVDRAEAERDVLFPGYTHLQRAQPIRWSHWILSHAVALTRDSERLLEVRKRINVLPLGSGAIAGNPLGVDRELLRAELNFGAITLNSMDATSERDFVAEFLFWRSLCMTHLSRMAEDLILYCTKEFSFVQLSDAYSTGSSLMPRKKNPDSLELIRSKAGRVFGRCAGLLMTLKGLPSTYNKDLQEDKEAVFEVSDTMSAVLQVATGVISTLQIHQENMGQALSPDMLATDLAYYLVRKGMPFRQAHEASGKAVFMAETKGVALNQLSLQELQTISPLFSGDVICVWDYRHSVEQYGALGGTARSSVDWQIRQVRALLQAQQA'

In [90]:
list(all_data.keys())[0]

'A'

In [91]:
all_data[list(all_data.keys())[0]][1][0]

20

In [18]:
seq_B

'IMEKFNASIAYDRHLWEVDVQGSKAYSRGLEKAGLLTKAEMDQILHGLDKVKLNSNDEDIHTANERRLKELIGATAGKLHTGRSRNDQVVTDLRLWMRQTCSTLSGLLWELIRTMVDRAEAERDVLFPGYTHLQRAQPIRWSHWILSHAVALTRDSERLLEVRKRINVLPLGSGAIAGNPLGVDRELLRAELNFGAITLNSMDATSERDFVAEFLFWRSLCMTHLSRMAEDLILYCTKEFSFVQLSDAYSTGSSLMPQKKNPDSLELIRSKAGRVFGRCAGLLMTLKGLPSTYNKDLQEDKEAVFEVSDTMSAVLQVATGVISTLQIHQENMGQALSPDMLATDLAYYLVRKGMPFRQAHEASGKAVFMAETKGVALNQLSLQELQTISPLFSGDVICVWDYRHSVEQYGALGGTARSSVDWQIRQVRALLQAQ'

In [20]:
if seq_A==seq_B:
    print('yes')

yes


In [34]:
meta_data, dataframe = read_from_prism('/sbinlab/panf/Cartesian_pipeline_runs/Nicolas/1aos/prism_pdb_XXX_1aos_A.txt')
prism_seq = meta_data['protein']['sequence']
prism_var = {}
for var in dataframe['variant'].tolist():
    resi = int(var[1:-1]) # Residue number from variant in prism file
    if resi in prism_var.keys():
        prism_var[resi].append(var)
    else:
        prism_var[resi] = [var]
        
if 'first_res_num' in meta_data['protein']:
    sys.exit('We have a first res num case! Needs to be handled - let Johanna know ;-)')

for key in prism_var.keys():
    native = f"{prism_var[key][0][0]}{key}{prism_var[key][0][0]}"
    if not native in prism_var[key]:
        prism_var[key].append(native) #results in dictionary of resi: list of variants 

In [94]:
alignments1 = pairwise2.align.globalms(prism_seq, seq_A, 2, -1, -0.5, -0.1)[0]
alignments2 = pairwise2.align.globalms(prism_seq, seq_B, 2, -1, -0.5, -0.1)[0]


alignments1_nresi = [[-1, '-']]*len(alignments1[1])
i = 0
for index, elem in enumerate(alignments1[1]):
    if elem !='-':
        alignments1_nresi[index] = [all_data[nchains[0]][1][i], all_data[nchains[0]][0][i]]
        i+=1
alignments2_nresi = [[-1, '-']]*len(alignments2[1])
i = 0
for index, elem in enumerate(alignments2[1]):
    if elem !='-':
        alignments2_nresi[index] = [all_data[nchains[1]][1][i], all_data[nchains[1]][0][i]]
        i+=1


vari_list = []
i=all_data[list(all_data.keys())[0]][1][0]
for index, res in enumerate(alignments1[1], start=i):
    if (alignments1[1][index-i] != '-') & (alignments1[1][index-i] == alignments2[1][index-i]):
        if index in prism_var.keys():
            for variant in prism_var[index]:
                mut1 = f"{alignments1_nresi[index-i][1]}{alignments1_nresi[index-i][0]}{variant[-1]}"
                mut2 = f"{alignments2_nresi[index-i][1]}{alignments2_nresi[index-i][0]}{variant[-1]}"
                merged_variant = f"{mut1}:{mut2}"
                vari_list.append(merged_variant)

In [120]:
df = pd.DataFrame(data=vari_list, columns=['variant'])
df['n_mut'] = 2
df['aa_ref'] = df['variant'].apply(lambda x: [resi[0] for resi in x.split(':')])
df['resi'] = df['variant'].apply(lambda x: [int(resi[1:-1]) for resi in x.split(':')])
df['aa_var'] = df['variant'].apply(lambda x: [resi[-1] for resi in x.split(':')])
df = df[['variant', 'aa_ref', 'resi', 'aa_var', 'n_mut']]

In [115]:
df

,variant,aa_ref,resi,aa_var,n_mut,sorter
0,I20=:I20=,"[I, I]","[20, 20]","[=, =]",2,20
1,I20A:I20A,"[I, I]","[20, 20]","[A, A]",2,20
2,I20C:I20C,"[I, I]","[20, 20]","[C, C]",2,20
3,I20D:I20D,"[I, I]","[20, 20]","[D, D]",2,20
4,I20E:I20E,"[I, I]","[20, 20]","[E, E]",2,20
...,...,...,...,...,...,...
9109,Q462T:Q462T,"[Q, Q]","[462, 462]","[T, T]",2,462
9110,Q462V:Q462V,"[Q, Q]","[462, 462]","[V, V]",2,462
9111,Q462W:Q462W,"[Q, Q]","[462, 462]","[W, W]",2,462
9112,Q462Y:Q462Y,"[Q, Q]","[462, 462]","[Y, Y]",2,462


In [134]:
def prepare_df(in_df, inclWT=True, drop_multi_mut=False, resisatu=False):
    df = in_df.copy()
    if drop_multi_mut:
        df = df[df['n_mut']==1]
    # drop WT variants and deletions
    df['drop'] = df['aa_var'].apply(lambda x: "".join(x) )
    drop = df[df['drop'].str.contains('~')].index
    df= df.drop(drop)

    df['resi_comb'] = df['resi'].apply(lambda x: ";".join([str(elem) for elem in x]))
    df['aa_wt_comb'] = df['aa_ref'].apply(lambda x: ";".join(x))
    df['aa_var_comb'] = df['aa_var'].apply(lambda x: ";".join(x))
    df['count'] = 1
    #df['sorter'] = df['resi'].str[0]

    
    if not inclWT:
        drop = df[df['drop'].str.contains('=')].index
        df= df.drop(drop)
        drop = df[df['aa_var'] == df['aa_ref']].index
        df= df.drop(drop)
        drops = df.copy()
    else:
        drops = df.copy()
        drop = drops[drops['drop'].str.contains('=')].index
        drops= drops.drop(drop)
        drop = drops[drops['aa_var'] == drops['aa_ref']].index
        drops= drops.drop(drop)
        df = drops.copy()
    if len(drops) !=0:
        drops = drops[['variant', 'resi_comb', 'aa_wt_comb', 'count']]
        drops = drops.groupby(['resi_comb', 'aa_wt_comb'],as_index=False).agg(lambda x : x.sum() if x.dtype in ['float64', 'int'] else ';'.join(x))
        drops['sum_muts'] = drops['variant'].apply(lambda x: len([item for subl in [i.split(':') for i in x.split(';')] for item in subl]))
        drops['sum_incl_wt'] = drops['sum_muts'] + drops['sum_muts']/drops['count']

    if len(df) == 0:
        logger.warning('No variants to process. Does it contain only WT and function switched on?')
        return
    df = df[['variant', 'resi_comb', 'aa_wt_comb', 'count']]
    df = df.groupby(['resi_comb', 'aa_wt_comb'],as_index=False).agg(lambda x : x.sum() if x.dtype in ['float64', 'int'] else ';'.join(x))
    df['sum_muts'] = df['variant'].apply(lambda x: len([item for subl in [i.split(':') for i in x.split(';')] for item in subl]))
    df['sum_incl_wt'] = df['sum_muts'] + df['sum_muts']/df['count']

    if resisatu:
        print(df)
        df['variant'] = df.apply(lambda x: make_satu(x['aa_wt_comb'], x['resi_comb'], inclWT=inclWT), axis=1)
        if inclWT:
            df['count'] = 20
            df['sum_muts'] = 20
            df['sum_incl_wt'] = 20.0
        else:
            df['count'] = 19
            df['sum_muts'] = 19
            df['sum_incl_wt'] = 20.0

    df['sorter'] = pd.to_numeric(df['resi_comb'].str.split(';').str[0])
    df.sort_values(by=['sorter'], inplace=True)

    return df, drops

In [136]:
prep_df, drops = prepare_df(df, inclWT=True, drop_multi_mut=False, resisatu=False)

In [ ]:
I20A:I20A;I20C:I20C;I20D:I20D;I20E:I20E;I20F:I20F;I20G:I20G;I20H:I20H;I20K:I20K;I20L:I20L;I20M:I20M;I20N:I20N;I20P:I20P;I20Q:I20Q;I20R:I20R;I20S:I20S;I20T:I20T;I20V:I20V;I20W:I20W;I20Y:I20Y
[['I20A', 'I20A'], ['I20C', 'I20C'], ['I20D', 'I20D'], ['I20E', 'I20E'], ['I20F', 'I20F'], ['I20G', 'I20G'], ['I20H', 'I20H'], ['I20K', 'I20K'], ['I20L', 'I20L'], ['I20M', 'I20M'], ['I20N', 'I20N'], ['I20P', 'I20P'], ['I20Q', 'I20Q'], ['I20R', 'I20R'], ['I20S', 'I20S'], ['I20T', 'I20T'], ['I20V', 'I20V'], ['I20W', 'I20W'], ['I20Y', 'I20Y']]
0 I20A
1 I20A

In [146]:
list(all_data.keys())[0]

'A'

In [151]:
for a, (b,c)  in zip([0,1,2], enumerate(['a', 'b', 'c'])):
    print(a, b, c)

0 0 a
1 1 b
2 2 c


In [155]:
import os
os.getcwd()

'/lustre/hpc/sbinlab/panf/PRISM/software/scripts'

In [153]:
for i, row in prep_df.iterrows():
    outstring = row['variant']
    mutant_line = [variants.split(':') for variants in row['variant'].split(';')]
    resstring = []
    for chain, (resind, residue) in zip(list(all_data.keys()), enumerate(mutant_line[0])):
        aa_ref = residue[0]
        res = residue[1:-1]
        muts = "".join([ mutant_line[l][resind][-1] for l in range(len(mutant_line)) if mutant_line[l][resind][-1] in AA])
        muts = aa_ref+muts
        resstring.append(f'{aa_ref} {chain}{res} {muts}')
    print(f"{' '.join(resstring)}\n")

I A20 IACDEFGHKLMNPQRSTVWY I B20 IACDEFGHKLMNPQRSTVWY

M A21 MACDEFGHIKLNPQRSTVWY M B21 MACDEFGHIKLNPQRSTVWY

E A22 EACDFGHIKLMNPQRSTVWY E B22 EACDFGHIKLMNPQRSTVWY

K A23 KACDEFGHILMNPQRSTVWY K B23 KACDEFGHILMNPQRSTVWY

F A24 FACDEGHIKLMNPQRSTVWY F B24 FACDEGHIKLMNPQRSTVWY

N A25 NACDEFGHIKLMPQRSTVWY N B25 NACDEFGHIKLMPQRSTVWY

A A26 ACDEFGHIKLMNPQRSTVWY A B26 ACDEFGHIKLMNPQRSTVWY

S A27 SACDEFGHIKLMNPQRTVWY S B27 SACDEFGHIKLMNPQRTVWY

I A28 IACDEFGHKLMNPQRSTVWY I B28 IACDEFGHKLMNPQRSTVWY

A A29 ACDEFGHIKLMNPQRSTVWY A B29 ACDEFGHIKLMNPQRSTVWY

Y A30 YACDEFGHIKLMNPQRSTVW Y B30 YACDEFGHIKLMNPQRSTVW

D A31 DACEFGHIKLMNPQRSTVWY D B31 DACEFGHIKLMNPQRSTVWY

R A32 RACDEFGHIKLMNPQSTVWY R B32 RACDEFGHIKLMNPQSTVWY

H A33 HACDEFGIKLMNPQRSTVWY H B33 HACDEFGIKLMNPQRSTVWY

L A34 LACDEFGHIKMNPQRSTVWY L B34 LACDEFGHIKMNPQRSTVWY

W A35 WACDEFGHIKLMNPQRSTVY W B35 WACDEFGHIKLMNPQRSTVY

E A36 EACDFGHIKLMNPQRSTVWY E B36 EACDFGHIKLMNPQRSTVWY

V A37 VACDEFGHIKLMNPQRSTWY V B37 VACDEFGHIKLMNPQRSTWY

D A38 DACE

In [ ]:
'I 100 IACDEFGHKMNPQRSTVWYL I 100 IACDEFGHKMNPQRSTVWYL'